In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json
import torch, numpy as np
import argparse

# Include prompt creation helper functions
from utils.prompt_utils import *
from utils.intervention_utils import *
from utils.model_utils import *
from utils.eval_utils import *
from utils.extract_utils import *
from compute_indirect_effect import compute_indirect_effect

In [14]:
dataset_name = "antonym"
model_name = "../flan-llama-7b/"
root_data_dir = "../dataset_files/"
save_path_root = "../debug"
ie_path_root = "../results/INS/CIE/flan-llama-7b/Prompt1/antonym"
seed = 42
device = "cuda"
mean_activations_path = "../results/INS/CIE/flan-llama-7b/Prompt1/antonym/antonym_mean_head_activations.pt"
indirect_effect_path = "../results/INS/CIE/flan-llama-7b/Prompt1/antonym/antonym_indirect_effect.pt"
n_top_heads = 10
eval_edit_layer = -1

test_split = float(0.3)
n_shots = 0
n_trials = 25

prefixes = {"input": "Input: ", "output": "Output:", "instructions": "Instruction: Output the antonym of the given word."}
separators = {"input": "\n", "output": "\n", "instructions": "\n"}
no_instruction = True

compute_baseline = True

generate_str = False
metric = "exact_match_score"
universal_set = False

In [11]:
torch.set_grad_enabled(False)
print("Loading Model")
model, tokenizer, model_config = load_gpt_model_and_tokenizer(model_name, device=device)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading Model
Loading:  ../flan-llama-7b/


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
if eval_edit_layer == -1: # sweep over all layers if edit_layer=-1
    eval_edit_layer = [0, model_config['n_layers']]

 # Load the dataset
print("Loading Dataset")
set_seed(seed)
dataset = load_dataset(dataset_name, root_data_dir=root_data_dir, test_size=test_split, seed=seed)

if not os.path.exists(save_path_root):
    os.makedirs(save_path_root)

Loading Dataset


In [18]:
set_seed(seed+42)
fs_results_validation = n_shot_eval_no_intervention(dataset=dataset, n_shots=n_shots, model=model, model_config=model_config, tokenizer=tokenizer, prefixes=prefixes, separators=separators, compute_ppl=True, test_split='valid')
filter_set_validation = np.where(np.array(fs_results_validation['clean_rank_list']) == 0)[0]
set_seed(seed)
fs_results = n_shot_eval_no_intervention(dataset=dataset, n_shots=n_shots, model=model, model_config=model_config, tokenizer=tokenizer, prefixes=prefixes, separators=separators, compute_ppl=True)
filter_set = np.where(np.array(fs_results['clean_rank_list']) == 0)[0]

100%|██████████| 504/504 [00:14<00:00, 35.61it/s]


In [20]:
if no_instruction:
    prefixes["instructions"] = ""
    separators["instructions"] = ""


In [ ]:
mean_activations = torch.load(mean_activations_path)
indirect_effect = torch.load(indirect_effect_path)